# Required import 

In [1]:
import SimpleITK as sitk
import os
import glob
import numpy as np
import pydicom
import nibabel as nib
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import skimage
from skimage import data, measure
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import pickle
import torch
%matplotlib inline

# Loading images

In [2]:
data_dir=r'C:\GAINED\resampled_croped'

In [3]:
def get_path_modality_and_mask(data_dir):
    
    ls_idx_pet = list(map(lambda x:x.split('\\')[-1].split('_')[0] , glob.glob(data_dir +'\\'+ 'PET0\\*00001.nii*')))    
    ls_idx_mask = list(np.unique(np.array(list(map(lambda x:x.split('\\')[-1][:14], glob.glob(data_dir +'\\'+'PET0_mask*/*nii*'))))))
    #print(ls_idx_mask)
    #print(ls_idx_pet)
    ls_ids = list(set(ls_idx_pet).intersection(set(ls_idx_mask)))
        
    pt_path=[os.path.join(data_dir,'PET0',ids+'_00001.nii') for ids in ls_ids]
    
    mask_path=[os.path.join(data_dir,'PET0_masks',ids+'_mask.nii') for ids in ls_ids]
    
    
    return pt_path,mask_path

In [4]:
pt_path,mask_path=get_path_modality_and_mask(data_dir)

In [5]:
all_mask_img=[]
for path in mask_path[:100]:
    mask_sitk=sitk.ReadImage(path)
    masks_np=sitk.GetArrayFromImage(mask_sitk)
    for i in range(masks_np.shape[1]):
        all_mask_img.append(masks_np[:,i,:])
        
    
all_pt_img=[]
for path in pt_path[:100]:
    img_sitk=sitk.ReadImage(path)
    img_np=sitk.GetArrayFromImage(img_sitk)
    for i in range(img_np.shape[1]):
        all_pt_img.append(img_np[:,i,:])
        
all_pt_img=np.array(all_pt_img)
all_mask_img=np.array(all_mask_img)

In [ ]:
# utilities function

In [12]:
# @hidden_cell
def get_one_hot_encoding(y, n_classes):
    """
    transform a numpy label array to a one-hot array of the same shape.
    :param y: array of shape (b, 1, y, x, (z)).
    :param n_classes: int, number of classes to unfold in one-hot encoding.
    :return y_ohe: array of shape (b, n_classes, y, x, (z))
    """
    dim = len(y.shape) - 2
    if dim == 2:
        y_ohe = np.zeros((y.shape[0], n_classes, y.shape[2], y.shape[3])).astype('int32')
    if dim ==3:
        y_ohe = np.zeros((y.shape[0], n_classes, y.shape[2], y.shape[3], y.shape[4])).astype('int32')
    for cl in range(n_classes):
        y_ohe[:, cl][y[:, 0] == cl] = 1
    return y_ohe

In [7]:
def get_dice_per_batch_and_class(pred, y, n_classes):
    '''
    computes dice scores per batch instance and class.
    :param pred: prediction array of shape (b, 1, y, x, (z)) (e.g. softmax prediction with argmax over dim 1)
    :param y: ground truth array of shape (b, 1, y, x, (z)) (contains int [0, ..., n_classes]
    :param n_classes: int
    :return: dice scores of shape (b, c)
    '''
    pred = (pred>=1).astype(np.int)
    y = (y>=1).astype(np.int)
    pred = get_one_hot_encoding(pred, n_classes)
    y = get_one_hot_encoding(y, n_classes)
    axes = tuple(range(2, len(pred.shape)))
    intersect = np.sum(pred*y, axis=axes)
    denominator = np.sum(pred, axis=axes)+np.sum(y, axis=axes) + 1e-8
    dice = 2.0*intersect / denominator
    return dice